In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("transformations").getOrCreate()

In [4]:
! hadoop fs -ls /user/tkm/retail_db

Found 6 items
-rw-r--r--   1 TARUN supergroup       1029 2024-05-06 13:31 /user/tkm/retail_db/categories.csv
-rw-r--r--   1 TARUN supergroup     953719 2024-05-06 13:31 /user/tkm/retail_db/customers.csv
-rw-r--r--   1 TARUN supergroup         60 2024-05-06 13:31 /user/tkm/retail_db/departments.csv
-rw-r--r--   1 TARUN supergroup    5408880 2024-05-06 13:31 /user/tkm/retail_db/order_items.csv
-rw-r--r--   1 TARUN supergroup    2999944 2024-05-06 13:31 /user/tkm/retail_db/orders.csv
-rw-r--r--   1 TARUN supergroup     174155 2024-05-06 13:31 /user/tkm/retail_db/products.csv


In [5]:
! hadoop fs -head /user/tkm/retail_db/order_items.csv

1,1,957,1,299.98,299.98
2,2,1073,1,199.99,199.99
3,2,502,5,250.0,50.0
4,2,403,1,129.99,129.99
5,4,897,2,49.98,24.99
6,4,365,5,299.95,59.99
7,4,502,3,150.0,50.0
8,4,1014,4,199.92,49.98
9,5,957,1,299.98,299.98
10,5,365,5,299.95,59.99
11,5,1014,2,99.96,49.98
12,5,957,1,299.98,299.98
13,5,403,1,129.99,129.99
14,7,1073,1,199.99,199.99
15,7,957,1,299.98,299.98
16,7,926,5,79.95,15.99
17,8,365,3,179.97,59.99
18,8,365,5,299.95,59.99
19,8,1014,4,199.92,49.98
20,8,502,1,50.0,50.0
21,9,191,2,199.98,99.99
22,9,1073,1,199.99,199.99
23,9,1073,1,199.99,199.99
24,10,1073,1,199.99,199.99
25,10,1014,2,99.96,49.98
26,10,403,1,129.99,129.99
27,10,917,1,21.99,21.99
28,10,1073,1,199.99,199.99
29,11,365,1,59.99,59.99
30,11,627,4,159.96,39.99
31,11,1014,1,49.98,49.98
32,11,191,4,399.96,99.99
33,11,1014,5,249.9,49.98
34,12,957,1,299.98,299.98
35,12,134,4,100.0,25.0
36,12,1014,3,149.94,49.98
37,12,191,5,499.95,99.99
38,12,502,5,250.0,50.0
39,13,276,4,127.96,31.99
40,14,1004,1,399.98,399.98
41,14,1014,2,99.96,49.

In [6]:
df = spark.read \
    .format("csv") \
        .option("inferSchema","true") \
            .load("hdfs://localhost:9000/user/tkm/retail_db/order_items.csv")
        

In [7]:
df.show(3)

+---+---+----+---+------+------+
|_c0|_c1| _c2|_c3|   _c4|   _c5|
+---+---+----+---+------+------+
|  1|  1| 957|  1|299.98|299.98|
|  2|  2|1073|  1|199.99|199.99|
|  3|  2| 502|  5| 250.0|  50.0|
+---+---+----+---+------+------+
only showing top 3 rows



In [8]:
df1 = df.toDF("item_id","order_id","product_id","quantity","subtotal","product_price")

In [10]:
df1.show(3)

+-------+--------+----------+--------+--------+-------------+
|item_id|order_id|product_id|quantity|subtotal|product_price|
+-------+--------+----------+--------+--------+-------------+
|      1|       1|       957|       1|  299.98|       299.98|
|      2|       2|      1073|       1|  199.99|       199.99|
|      3|       2|       502|       5|   250.0|         50.0|
+-------+--------+----------+--------+--------+-------------+
only showing top 3 rows



In [11]:
df2 = df1.drop("subtotal")

In [14]:
df2.show(3)

+-------+--------+----------+--------+-------------+
|item_id|order_id|product_id|quantity|product_price|
+-------+--------+----------+--------+-------------+
|      1|       1|       957|       1|       299.98|
|      2|       2|      1073|       1|       199.99|
|      3|       2|       502|       5|         50.0|
+-------+--------+----------+--------+-------------+
only showing top 3 rows



In [16]:
df2.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- product_price: double (nullable = true)



In [17]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [18]:
df3 = df2.withColumn("subtotal",col("quantity")*col("product_price"))

#col("quantity") = df2["quantity"] = df2.quantity 

In [22]:
df2.withColumn("subtotal",df2["quantity"]*df2["product_price"]).show(3)


+-------+--------+----------+--------+-------------+--------+
|item_id|order_id|product_id|quantity|product_price|subtotal|
+-------+--------+----------+--------+-------------+--------+
|      1|       1|       957|       1|       299.98|  299.98|
|      2|       2|      1073|       1|       199.99|  199.99|
|      3|       2|       502|       5|         50.0|   250.0|
+-------+--------+----------+--------+-------------+--------+
only showing top 3 rows



In [23]:
df2.withColumn("subtotal",df2.quantity*df2.product_price).show(3)

+-------+--------+----------+--------+-------------+--------+
|item_id|order_id|product_id|quantity|product_price|subtotal|
+-------+--------+----------+--------+-------------+--------+
|      1|       1|       957|       1|       299.98|  299.98|
|      2|       2|      1073|       1|       199.99|  199.99|
|      3|       2|       502|       5|         50.0|   250.0|
+-------+--------+----------+--------+-------------+--------+
only showing top 3 rows



In [33]:
df2.withColumn("subtotal",expr("quantity*product_price")).show(3)

+-------+--------+----------+--------+-------------+--------+
|item_id|order_id|product_id|quantity|product_price|subtotal|
+-------+--------+----------+--------+-------------+--------+
|      1|       1|       957|       1|       299.98|  299.98|
|      2|       2|      1073|       1|       199.99|  199.99|
|      3|       2|       502|       5|         50.0|   250.0|
+-------+--------+----------+--------+-------------+--------+
only showing top 3 rows



In [19]:
df3.show(3)

+-------+--------+----------+--------+-------------+--------+
|item_id|order_id|product_id|quantity|product_price|subtotal|
+-------+--------+----------+--------+-------------+--------+
|      1|       1|       957|       1|       299.98|  299.98|
|      2|       2|      1073|       1|       199.99|  199.99|
|      3|       2|       502|       5|         50.0|   250.0|
+-------+--------+----------+--------+-------------+--------+
only showing top 3 rows



In [24]:
df4 = df3.withColumnRenamed("subtotal","total")
df4.show(3)

+-------+--------+----------+--------+-------------+------+
|item_id|order_id|product_id|quantity|product_price| total|
+-------+--------+----------+--------+-------------+------+
|      1|       1|       957|       1|       299.98|299.98|
|      2|       2|      1073|       1|       199.99|199.99|
|      3|       2|       502|       5|         50.0| 250.0|
+-------+--------+----------+--------+-------------+------+
only showing top 3 rows



In [27]:
df2.select("*").show(3)

+-------+--------+----------+--------+-------------+
|item_id|order_id|product_id|quantity|product_price|
+-------+--------+----------+--------+-------------+
|      1|       1|       957|       1|       299.98|
|      2|       2|      1073|       1|       199.99|
|      3|       2|       502|       5|         50.0|
+-------+--------+----------+--------+-------------+
only showing top 3 rows



In [29]:
df2.select("product_id","product_price").show(3)

+----------+-------------+
|product_id|product_price|
+----------+-------------+
|       957|       299.98|
|      1073|       199.99|
|       502|         50.0|
+----------+-------------+
only showing top 3 rows



In [30]:
df2.select("*",expr("quantity * product_price as subtotal")).show(3)

+-------+--------+----------+--------+-------------+--------+
|item_id|order_id|product_id|quantity|product_price|subtotal|
+-------+--------+----------+--------+-------------+--------+
|      1|       1|       957|       1|       299.98|  299.98|
|      2|       2|      1073|       1|       199.99|  199.99|
|      3|       2|       502|       5|         50.0|   250.0|
+-------+--------+----------+--------+-------------+--------+
only showing top 3 rows



In [31]:
df2.selectExpr("*","quantity * product_price as subtotal").show(3)

+-------+--------+----------+--------+-------------+--------+
|item_id|order_id|product_id|quantity|product_price|subtotal|
+-------+--------+----------+--------+-------------+--------+
|      1|       1|       957|       1|       299.98|  299.98|
|      2|       2|      1073|       1|       199.99|  199.99|
|      3|       2|       502|       5|         50.0|   250.0|
+-------+--------+----------+--------+-------------+--------+
only showing top 3 rows



In [ ]:
raw_df = spark.

In [34]:
! hadoop fs -head /user/tkm/retail_db/products.csv

1,2,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,,59.98,http://images.acmesports.sports/Quest+Q64+10+FT.+x+10+FT.+Slant+Leg+Instant+Up+Canopy
2,2,Under Armour Men's Highlight MC Football Clea,,129.99,http://images.acmesports.sports/Under+Armour+Men%27s+Highlight+MC+Football+Cleat
3,2,Under Armour Men's Renegade D Mid Football Cl,,89.99,http://images.acmesports.sports/Under+Armour+Men%27s+Renegade+D+Mid+Football+Cleat
4,2,Under Armour Men's Renegade D Mid Football Cl,,89.99,http://images.acmesports.sports/Under+Armour+Men%27s+Renegade+D+Mid+Football+Cleat
5,2,Riddell Youth Revolution Speed Custom Footbal,,199.99,http://images.acmesports.sports/Riddell+Youth+Revolution+Speed+Custom+Football+Helmet
6,2,Jordan Men's VI Retro TD Football Cleat,,134.99,http://images.acmesports.sports/Jordan+Men%27s+VI+Retro+TD+Football+Cleat
7,2,Schutt Youth Recruit Hybrid Custom Football H,,99.99,http://images.acmesports.sports/Schutt+Youth+Recruit+Hybrid+Custom+Football+Helmet+2014
8,2,Nike Men's Vapor Ca

In [35]:
raw_df = spark.read \
    .format("csv") \
        .option("inferSchema","true") \
            .load("hdfs://localhost:9000/user/tkm/retail_db/products.csv")

In [36]:
raw_df.show(4)

+---+---+--------------------+----+------+--------------------+
|_c0|_c1|                 _c2| _c3|   _c4|                 _c5|
+---+---+--------------------+----+------+--------------------+
|  1|  2|Quest Q64 10 FT. ...|null| 59.98|http://images.acm...|
|  2|  2|Under Armour Men'...|null|129.99|http://images.acm...|
|  3|  2|Under Armour Men'...|null| 89.99|http://images.acm...|
|  4|  2|Under Armour Men'...|null| 89.99|http://images.acm...|
+---+---+--------------------+----+------+--------------------+
only showing top 4 rows



In [37]:
product_df = raw_df.toDF("product_id","category_id","product_name","description","price","image")
product_df.show(3)

+----------+-----------+--------------------+-----------+------+--------------------+
|product_id|category_id|        product_name|description| price|               image|
+----------+-----------+--------------------+-----------+------+--------------------+
|         1|          2|Quest Q64 10 FT. ...|       null| 59.98|http://images.acm...|
|         2|          2|Under Armour Men'...|       null|129.99|http://images.acm...|
|         3|          2|Under Armour Men'...|       null| 89.99|http://images.acm...|
+----------+-----------+--------------------+-----------+------+--------------------+
only showing top 3 rows



In [45]:
product_df.withColumn("price",expr("price*1.2")).show(3)

+----------+-----------+--------------------+-----------+------------------+--------------------+
|product_id|category_id|        product_name|description|             price|               image|
+----------+-----------+--------------------+-----------+------------------+--------------------+
|         1|          2|Quest Q64 10 FT. ...|       null|            71.976|http://images.acm...|
|         2|          2|Under Armour Men'...|       null|           155.988|http://images.acm...|
|         3|          2|Under Armour Men'...|       null|107.98799999999999|http://images.acm...|
+----------+-----------+--------------------+-----------+------------------+--------------------+
only showing top 3 rows



In [46]:
product_df.show()

+----------+-----------+--------------------+-----------+------+--------------------+
|product_id|category_id|        product_name|description| price|               image|
+----------+-----------+--------------------+-----------+------+--------------------+
|         1|          2|Quest Q64 10 FT. ...|       null| 59.98|http://images.acm...|
|         2|          2|Under Armour Men'...|       null|129.99|http://images.acm...|
|         3|          2|Under Armour Men'...|       null| 89.99|http://images.acm...|
|         4|          2|Under Armour Men'...|       null| 89.99|http://images.acm...|
|         5|          2|Riddell Youth Rev...|       null|199.99|http://images.acm...|
|         6|          2|Jordan Men's VI R...|       null|134.99|http://images.acm...|
|         7|          2|Schutt Youth Recr...|       null| 99.99|http://images.acm...|
|         8|          2|Nike Men's Vapor ...|       null|129.99|http://images.acm...|
|         9|          2|Nike Adult Vapor ...|       nu

In [53]:
product_df.withColumn("price",expr(" price*10 ")).show(5)

+----------+-----------+--------------------+-----------+------+--------------------+
|product_id|category_id|        product_name|description| price|               image|
+----------+-----------+--------------------+-----------+------+--------------------+
|         1|          2|Quest Q64 10 FT. ...|       null| 599.8|http://images.acm...|
|         2|          2|Under Armour Men'...|       null|1299.9|http://images.acm...|
|         3|          2|Under Armour Men'...|       null| 899.9|http://images.acm...|
|         4|          2|Under Armour Men'...|       null| 899.9|http://images.acm...|
|         5|          2|Riddell Youth Rev...|       null|1999.9|http://images.acm...|
+----------+-----------+--------------------+-----------+------+--------------------+
only showing top 5 rows



In [51]:
product_df.withColumn("new_price",
                      expr("CASE WHEN product_name like '%Nike%' THEN price*10 \
                           WHEN product_name like '%Armour%' THEN price*100 \
                           ELSE price \
                           END")).show(10)

+----------+-----------+--------------------+-----------+------+--------------------+---------+
|product_id|category_id|        product_name|description| price|               image|new_price|
+----------+-----------+--------------------+-----------+------+--------------------+---------+
|         1|          2|Quest Q64 10 FT. ...|       null| 59.98|http://images.acm...|    59.98|
|         2|          2|Under Armour Men'...|       null|129.99|http://images.acm...|  12999.0|
|         3|          2|Under Armour Men'...|       null| 89.99|http://images.acm...|   8999.0|
|         4|          2|Under Armour Men'...|       null| 89.99|http://images.acm...|   8999.0|
|         5|          2|Riddell Youth Rev...|       null|199.99|http://images.acm...|   199.99|
|         6|          2|Jordan Men's VI R...|       null|134.99|http://images.acm...|   134.99|
|         7|          2|Schutt Youth Recr...|       null| 99.99|http://images.acm...|    99.99|
|         8|          2|Nike Men's Vapor

In [54]:
product_df.select("*",
                      expr("CASE WHEN product_name like '%Nike%' THEN price*10 \
                           WHEN product_name like '%Armour%' THEN price*100 \
                           ELSE price \
                           END \
                            AS new_price")).show(10)

+----------+-----------+--------------------+-----------+------+--------------------+---------+
|product_id|category_id|        product_name|description| price|               image|new_price|
+----------+-----------+--------------------+-----------+------+--------------------+---------+
|         1|          2|Quest Q64 10 FT. ...|       null| 59.98|http://images.acm...|    59.98|
|         2|          2|Under Armour Men'...|       null|129.99|http://images.acm...|  12999.0|
|         3|          2|Under Armour Men'...|       null| 89.99|http://images.acm...|   8999.0|
|         4|          2|Under Armour Men'...|       null| 89.99|http://images.acm...|   8999.0|
|         5|          2|Riddell Youth Rev...|       null|199.99|http://images.acm...|   199.99|
|         6|          2|Jordan Men's VI R...|       null|134.99|http://images.acm...|   134.99|
|         7|          2|Schutt Youth Recr...|       null| 99.99|http://images.acm...|    99.99|
|         8|          2|Nike Men's Vapor

In [56]:
product_df.selectExpr("*",
                  "CASE WHEN product_name like '%Nike%' THEN price*10 \
                    WHEN product_name like '%Armour%' THEN price*100 \
                    ELSE price \
                    END \
                    AS new_price").show(10)

+----------+-----------+--------------------+-----------+------+--------------------+---------+
|product_id|category_id|        product_name|description| price|               image|new_price|
+----------+-----------+--------------------+-----------+------+--------------------+---------+
|         1|          2|Quest Q64 10 FT. ...|       null| 59.98|http://images.acm...|    59.98|
|         2|          2|Under Armour Men'...|       null|129.99|http://images.acm...|  12999.0|
|         3|          2|Under Armour Men'...|       null| 89.99|http://images.acm...|   8999.0|
|         4|          2|Under Armour Men'...|       null| 89.99|http://images.acm...|   8999.0|
|         5|          2|Riddell Youth Rev...|       null|199.99|http://images.acm...|   199.99|
|         6|          2|Jordan Men's VI R...|       null|134.99|http://images.acm...|   134.99|
|         7|          2|Schutt Youth Recr...|       null| 99.99|http://images.acm...|    99.99|
|         8|          2|Nike Men's Vapor

In [ ]:
spark.stop()